In [4]:
import csv
import os
import requests as req
import json
from collections import Counter

In [5]:
#Donorschoose.org API
api_key = "DONORSCHOOSE"
url = "https://api.donorschoose.org/common/json_feed.html?max=100"

In [6]:
#Open a CSV for a list of Bay Area cities
with open("bayarea_cities.csv") as cities:
    cities_reader = csv.reader(cities)
    
    cities = [city[0] for city in cities_reader]
    
    print(len(cities))

cities = cities[6:13]
print(cities)

102
['Belmont', 'Belvedere', 'Benicia', 'Berkeley', 'Brentwood', 'Brisbane', 'Burlingame']


In [7]:
# Build partial query URL
query_url = url + "&APIkey=" + api_key + "&state=CA" + "&cityName="
#location_data = [req.get(query_url + city).json() for city in cities]

all_data = {}

for city in cities:
    
    results = req.get(query_url + city).json()
    all_data[city] = results   

print(json.dumps(all_data, indent=2))

{
  "Belmont": {
    "searchTerms": "CA",
    "searchURL": "https://www.donorschoose.org/donors/search.html?APIkey=DONORSCHOOSE&cityName=Belmont&state=CA&utm_source=api&utm_medium=feed&utm_content=searchlink&utm_campaign=DONORSCHOOSE",
    "totalProposals": "10",
    "index": "0",
    "max": "10",
    "breadcrumb": [
      [
        "max",
        "100",
        ""
      ],
      [
        "state",
        "CA",
        "California"
      ]
    ],
    "proposals": [
      {
        "id": "2774329",
        "hashCode": "1835973610",
        "proposalURL": "https://www.donorschoose.org/project/help-us-create-great-music-with-a-cello/2774329/?verify=1835973610&utm_source=api&utm_medium=feed&utm_content=bodylink&utm_campaign=DONORSCHOOSE",
        "fundURL": "https://secure.donorschoose.org/donors/givingCart.html?proposalid=2774329&donationAmount=45&verify=1835973610&utm_source=api&utm_medium=feed&utm_content=fundlink&utm_campaign=DONORSCHOOSE",
        "imageURL": "https://storage.donorsc

In [8]:
city_data = {}

for cityname, data in all_data.items():
        
    #step 1: get cost to complete
    #step 2: get povertyType label
    #step 3: add this to the output vairable
    
    costcomplete = []
    povertytype = []
    
    data["proposals"]
    
    for x in data["proposals"]:
        
        costcomplete.append(float(x["costToComplete"]))
        povertytype.append(x["povertyType"]["label"])
    
    city_data[cityname] = {
        
        "costToComplete": sum(costcomplete),
        "povertyType": Counter(povertytype)
    }

city_data

{'Belmont': {'costToComplete': 5438.42, 'povertyType': Counter({'LOW': 10})},
 'Belvedere': {'costToComplete': 0, 'povertyType': Counter()},
 'Benicia': {'costToComplete': 7150.92,
  'povertyType': Counter({'LOW': 3, 'MODERATELOW': 5})},
 'Berkeley': {'costToComplete': 18144.340000000004,
  'povertyType': Counter({'MODERATE': 1,
           'MODERATEHIGH': 3,
           'MODERATELOW': 22})},
 'Brentwood': {'costToComplete': 11523.6,
  'povertyType': Counter({'LOW': 6, 'MODERATEHIGH': 2, 'MODERATELOW': 4})},
 'Brisbane': {'costToComplete': 0, 'povertyType': Counter()},
 'Burlingame': {'costToComplete': 1083.4, 'povertyType': Counter({'LOW': 3})}}

In [9]:
#list of cityname, costtocomplete, number of low, number of modeatelow, etc.

csv_data = []

citynames = sorted(city_data.keys())

for cityname in citynames: 
    
    data = city_data[cityname]
    
    one_row = []
    
    one_row.append(cityname)
    one_row.append(data['costToComplete'])  
    
    labeltype = ["LOW", "MODERATELOW", "MODERATE", "MODERATEHIGH", "HIGH", "HIGHEST", "UNKNOWN"]
    
    for x in labeltype:
        
        one_row.append(data['povertyType'].get(x, 0))
    
    csv_data.append(one_row)

csv_data

[['Belmont', 5438.42, 10, 0, 0, 0, 0, 0, 0],
 ['Belvedere', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Benicia', 7150.92, 3, 5, 0, 0, 0, 0, 0],
 ['Berkeley', 18144.340000000004, 0, 22, 1, 3, 0, 0, 0],
 ['Brentwood', 11523.6, 6, 4, 0, 2, 0, 0, 0],
 ['Brisbane', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Burlingame', 1083.4, 3, 0, 0, 0, 0, 0, 0]]

In [12]:
root_path = os.path.join(os.getcwd())
data_path = os.path.join(root_path)
csvpath = os.path.join(root_path, "donorschoose_api.csv")
with open(csvpath, "a", newline="") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=",")
    csvwriter.writerows(csv_data)